In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'sample_submission.csv', 'test.csv']


In [2]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_181"; OpenJDK Runtime Environment (build 1.8.0_181-8u181-b13-2~deb9u1-b13); OpenJDK 64-Bit Server VM (build 25.181-b13, mixed mode)
  Starting server from /opt/conda/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp0b9v28a1
  JVM stdout: /tmp/tmp0b9v28a1/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp0b9v28a1/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.6
H2O cluster version age:,16 days
H2O cluster name:,H2O_from_python_unknownUser_g26l2t
H2O cluster total nodes:,1
H2O cluster free memory:,3.484 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import math

from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch


import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

plt.style.use('seaborn')
sns.set(font_scale=1)

In [4]:
random_state = 82
np.random.seed(random_state)
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

In [5]:
X_train, X_val = train_test_split(df_train, test_size=0.2, random_state=random_state)

In [6]:
X_train_h2o = h2o.H2OFrame(X_train.drop(["ID_code"], axis=1))
X_train_h2o["target"]=X_train_h2o["target"].asfactor()    ## Converting Target Variable as Factor

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [7]:
X_val_h2o = h2o.H2OFrame(X_val.drop(["ID_code"], axis=1))
X_val_h2o["target"]=X_val_h2o["target"].asfactor()    ## Converting Target Variable as Factor

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [8]:
X_test = h2o.H2OFrame(df_test.drop(["ID_code"], axis=1))

Parse progress: |██████████████████████████████████████████████████████████████| 100%


In [18]:
hyper_params = {'max_depth' : list(range(11,20,1)),
#                 'sample_rate': [x/100. for x in range(20,101)],
#                 'col_sample_rate' : [x/100. for x in range(20,101)],
#                 'col_sample_rate_per_tree': [x/100. for x in range(20,101)],
#                 'col_sample_rate_change_per_level': [x/100. for x in range(90,111)],
                'min_rows': [2**x for x in range(0,int(math.log(X_train_h2o.nrow,2)-1)+1)],
                'nbins': [2**x for x in range(4,11)],
                'nbins_cats': [2**x for x in range(4,13)],
                'min_split_improvement': [0,1e-8,1e-6,1e-4],
                'histogram_type': ["UniformAdaptive","QuantilesGlobal","RoundRobin"]}
gbm = H2OGradientBoostingEstimator(
    ntrees = 1000, 
    
    distribution = "bernoulli",

    learn_rate = 0.02012,
    
    learn_rate_annealing=0.99,

    stopping_rounds = 7, 
    
    stopping_tolerance = 1e-4, 
    
    stopping_metric = "AUC",

#     sample_rate = 0.8,                                        

#     col_sample_rate = 0.8,
    
#     col_sample_rate_change_per_level = 0.999,

    seed = random_state,
    
#     calibrate_model = True,
    
#     calibration_frame = X_train_h2o,
    
    score_tree_interval = 10,
    
    nfolds = 3)

In [19]:
grid = H2OGridSearch(gbm,hyper_params,
                         grid_id = 'depth_grid',
                         search_criteria = {'strategy': "Cartesian"})

In [20]:
grid.train(x=X_train_h2o.names[1:],y=X_train_h2o.names[0],
           training_frame = X_train_h2o)

gbm Grid Build progress: |█ (cancelled)


H2OJobCancelled: Job<$03017f00000132d4ffffffff$_ae25570bddbf4f6bf77a49ca72e349b9> was cancelled by the user.

In [ ]:
sorted_grid = grid.get_grid(sort_by='auc',decreasing=True)
print(sorted_grid)

In [ ]:
model = h2o.get_model(sorted_grid.sorted_metric_table()['model_ids'][0])
performance_model = model.model_performance(X_val_h2o)
print(performance_model.auc())

In [ ]:
model.cross_validation_metrics_summary()

In [ ]:
model.varimp_plot()

In [ ]:
pred = model.predict(test_data=X_test)

In [ ]:
pred = pred.as_data_frame() 

In [ ]:
sub = pd.read_csv("../input/sample_submission.csv")

In [ ]:
sub["target"] = pred["predict"]

In [ ]:
sub.head()

In [ ]:
sub.to_csv("h2o_submission.csv", index=False)